In [35]:
import torch
from torch import nn

import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

In [36]:
# Download mnist fashion train and test datasets
mnist_train = datasets.FashionMNIST(root="./data", download=True, train=True, transform=ToTensor())
mnist_test = datasets.FashionMNIST(root="./data", download=False, train=True, transform=ToTensor())

In [37]:
from torch.utils.data import DataLoader

# Converting to dataloaders
train_dataloder = DataLoader(mnist_train, batch_size=32, shuffle=True)
test_dataloder = DataLoader(mnist_test, batch_size=32, shuffle=True)

In [38]:
# Defining the model
model = nn.Sequential(
    nn.Conv2d(1, 3, kernel_size=(3, 3), padding=1, padding_mode="reflect"), # CNN 2 dimensional layer with a 3x3 filter
    nn.MaxPool2d(kernel_size=2), # Max pooling to reduce the dimension and turn less complex the nn
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(588, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)

# Defining the loss function
criterion = nn.CrossEntropyLoss()

# Defining the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [39]:
# Reshaping the data (validation)
for X, y in train_dataloder:
    print(X.shape)
    print(y.shape)
    
    y = nn.functional.one_hot(y, num_classes=10).type(torch.float32)
    print(y.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])
torch.Size([32, 10])


In [40]:
for i in range(10):
    loss_sum = 0
    for X, y in train_dataloder:
        y = nn.functional.one_hot(y, num_classes=10).type(torch.float32)

        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    print(loss_sum)

963.3684431016445
660.1483805924654
587.415435180068
538.5746393799782
498.1113281920552
466.8011701218784
441.2674172986299
414.1109722740948
393.24650387838483
369.92314274422824


In [41]:
model.eval()
with torch.no_grad():
    accurate = 0
    total = 0
    for X, y in test_dataloder:
        y_pred = nn.functional.softmax(model(X), dim=1)
        correct_pred = (y == y_pred.max(dim=1).indices)

        total += correct_pred.size(0)
        accurate += correct_pred.type(torch.int).sum().item()

    print(f"Accuracy: {(accurate/total)*100:.2f} %")

Accuracy: 93.47 %
